In [4]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from absbox import Generic

perf = ("Pool"
        ,("Mortgage",{"CDR":0.15},{"CPR":0.0015},{"Rate":0.3,"Lag":4},None)
        ,None
        ,None
       )

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-06-30","closing":"2021-06-30","firstPay":"2021-07-31"
     ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":10000,"originRate":["fix",0.045],"originTerm":48
          ,"freq":"Monthly","type":"Even","originDate":"2021-05-01"}
          ,{"currentBalance":10000
          ,"currentRate":0.08
          ,"remainTerm":48
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}))
    ,(("A",{"balance":7000
             ,"rate":0.01
             ,"originBalance":7000
             ,"originRate":0.00
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.01}
             ,"bondType":{"Sequential":None}})
      ,("M",{"balance":2000
             ,"rate":0.02
             ,"originBalance":2000
             ,"originRate":0.0
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.02}
             ,"bondType":{"Sequential":None}
             })
      ,("E",{"balance":1000
             ,"rate":0.11
             ,"originBalance":1000
             ,"originRate":0.11
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.11}
             ,"bondType":{"Sequential":None}
             })
     )
    ,list()
    ,{"revolving":[
          ["accrueAndPayIntBySeq","acc01",["A","M","E"]] 
          ,["buyAsset",["Current|Defaulted",1.0,0],"acc01",None]
         ]
      ,"amortizing":[
          ["accrueAndPayIntBySeq","acc01",["A","M","E"]]
          ,['payPrinBySeq',"acc02",["A","M","E"]]
          ,['payIntResidual',"acc01","E"]
       ]
      ,"endOfCollection":[
          ["writeOff","E",{"formula":("curPoolCollection",None,"Defaults")}]
          ,["liqSupport", "Issuer", "account", "acc01"
            , {"formula":("factor"
                          ,("poolBalance",)
                          ,0.01)}]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc02"]]
    ,{"Issuer":
       {"type" : "Unlimited","start":"2021-06-15"}}
    ,None
    ,None
    ,{"BeforeDistribution":
       {"dealStatusChange":
        {"condition":["any",[">=","2025-07-31"]
                           ,[("cumPoolNetLossRate",),">",0.05]] 
        ,"effects":("newStatus","Amortizing")
        ,"status":False
        ,"curable":False}}
        }
    ,("PreClosing","Revolving")
    )

In [23]:
from absbox import API
#localAPI = API("https://absbox.org/api/dev",check=False)
localAPI = API("http://localhost:8081",check=False)


r = localAPI.run(test01
                ,runAssump = [("revolving"
                                ,["constant"
                                  ,["Mortgage"
                                    ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                                      ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                                      ,{"currentBalance":2200
                                      ,"currentRate":0.08
                                      ,"remainTerm":20
                                      ,"status":"current"}]]
                                ,perf)]
                ,poolAssump = ("Pool"
                              ,("Mortgage",{"CDR":0.07},None,{"Rate":0.5,"Lag":4},None)
                              ,None
                              ,None)
                ,read=True)

⠙

Warning Message from server:
Account acc02 has cash to be distributed
LiquidityProvider Issuer is not paid off

In [72]:
from absbox import prodDealsBy,setDealsBy
from lenses import lens

In [102]:
dealMap = prodDealsBy(test01 ,(lens.bonds[2][1]['rate'],[0.1,0.2]),guessKey=True)

In [103]:
r = localAPI.runStructs(dealMap
                ,nonPoolAssump = [("revolving"
                                ,["constant"
                                  ,["Mortgage"
                                    ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                                      ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                                      ,{"currentBalance":2200
                                      ,"currentRate":0.08
                                      ,"remainTerm":20
                                      ,"status":"current"}]]
                                ,perf)]
                ,poolAssump = ("Pool"
                              ,("Mortgage",{"CDR":0.07},None,{"Rate":0.5,"Lag":4},None)
                              ,None
                              ,None)
                ,read=True)

⠼

In [104]:
dealMap['bonds-2-1-rate:0.1'].bonds[-1]

('E',
 {'balance': 1000,
  'rate': 0.1,
  'originBalance': 1000,
  'originRate': 0.11,
  'startDate': '2020-01-03',
  'rateType': {'Fixed': 0.11},
  'bondType': {'Sequential': None}})

In [105]:
r['bonds-2-1-rate:0.1']['bonds']['E']

,balance,interest,principal,rate,cash,factor,memo
date,,,,,,,
2021-07-31,942.13,8.00,0,0.1,8.00,0.94213,"[<WriteOff:E,57.87>, <PayInt:E>]"
2021-08-31,883.96,7.50,0,0.1,7.50,0.88396,"[<WriteOff:E,58.17>, <PayInt:E>]"
2021-09-30,824.13,6.77,0,0.1,6.77,0.82413,"[<WriteOff:E,59.83>, <PayInt:E>]"
2021-10-31,766.10,6.50,0,0.1,6.50,0.76610,"[<WriteOff:E,58.03>, <PayInt:E>]"
2021-11-30,706.12,5.80,0,0.1,5.80,0.70612,"[<WriteOff:E,59.98>, <PayInt:E>]"
2021-12-31,646.29,5.48,0,0.1,5.48,0.64629,"[<WriteOff:E,59.83>, <PayInt:E>]"
2022-01-31,586.71,4.98,0,0.1,4.98,0.58671,"[<WriteOff:E,59.58>, <PayInt:E>]"
2022-02-28,527.59,4.04,0,0.1,4.04,0.52759,"[<WriteOff:E,59.12>, <PayInt:E>]"
2022-03-31,474.30,4.02,0,0.1,4.02,0.47430,"[<WriteOff:E,53.29>, <PayInt:E>]"


In [61]:
r['bonds--1-1-rate:0.2']['bonds']['E']

KeyError: 'bonds--1-1-rate:0.2'

In [19]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-06-30,9766.38,0.00,0.00,0.00,0.00,0.00,0.00,0.080000,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-31,9534.70,207.27,64.93,0.00,24.41,0.00,0.00,0.080000,None,None,207.27,0.00,0.0,24.41,0.00,0.00
2021-08-31,9542.82,206.74,63.39,0.00,24.63,0.00,0.00,0.080000,None,None,414.01,0.00,0.0,49.04,0.00,0.00
2021-09-30,9438.80,217.28,63.42,0.03,27.31,0.00,0.00,0.080000,None,None,620.21,0.03,0.0,73.07,0.00,0.00
2021-10-31,9328.47,223.20,62.73,0.03,27.52,12.91,12.91,0.080000,None,None,825.90,0.05,0.0,95.75,12.91,12.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-08-31,3.00,2.26,0.00,0.00,0.04,0.05,0.17,0.078895,None,None,9194.39,0.00,0.0,571.99,298.79,298.79
2026-09-30,1.50,1.48,0.00,0.00,0.02,0.02,0.11,0.077217,None,None,9194.39,0.00,0.0,571.99,298.79,298.79
2026-10-31,0.61,0.89,0.00,0.00,0.00,0.00,0.08,0.072606,None,None,9194.39,0.00,0.0,571.99,298.79,298.79


In [11]:
r.keys()

dict_keys(['_deal', 'bonds', 'fees', 'accounts', 'liqProvider', 'agg_accounts', 'pool', 'pricing', 'result'])

In [20]:
r['result']['status']

,Date,From,To
0,2021-06-30,PreClosing,Revolving
1,2025-07-31,Revolving,Amortizing


In [22]:
r['result']['status']

,Date,From,To
0,2021-06-30,PreClosing,Revolving
1,2023-03-31,Revolving,Amortizing


In [24]:
r['result']['status']

,Date,From,To
0,2021-06-30,PreClosing,Revolving
1,2023-09-30,Revolving,Amortizing
